Create Player Ratings

In [26]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [34]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/sta

In [4]:
player_stats[['Position_Group', 'Position_Sub_Group', 'Position']].value_counts().sort_index()

Position_Group  Position_Sub_Group  Position                
Back            Back-Pocket         Back Pocket Left            1130
                                    Back Pocket Right           1119
                Half-Back           Half Back Flank Left        1118
                                    Half Back Flank Right       1113
                Key-Back            Centre Half Back            1113
                                    Full Back                   1133
Forward         Forward-Pocket      Forward Pocket Left         1112
                                    Forward Pocket Right        1121
                Half-Forward        Half Forward Flank Left     1109
                                    Half Forward Flank Right    1099
                Key-Forward         Centre Half Forward         1104
                                    Full Forward                1143
Interchange     Interchange         Interchange                 4466
                                    Substi

Different Positions are expected to do different things

In [36]:
match = player_stats[player_stats['Match_ID'] == "202101_BrisbaneLions_Sydney"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/sta

In [40]:
team_for = match.groupby('Team').sum()[['Score', 'xScore', 'xT_created', 'xT_denied', 'xT_prevented', 'xT_received', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
team_for.columns = [x+"_F" for x in list(team_for)]
team_for

,Score_F,xScore_F,xT_created_F,xT_denied_F,xT_prevented_F,xT_received_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F
Team,,,,,,,,,,,,,,
Brisbane Lions,90,70.776025,-0.749615,0.221936,0.851522,-1.000685,19.236294,14.293043,16.395997,2.840297,100.570235,84.321734,87.444271,13.125965
Sydney,122,93.887604,0.106715,0.851522,0.221936,0.361493,24.260554,16.380475,22.310430,1.950125,121.078925,115.966759,114.265726,6.813198


In [41]:
team_against = match.groupby(['Opponent']).sum()[['Score', 'xScore', 'xT_created', 'xT_denied', 'xT_prevented', 'xT_received', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
team_against.columns = [x+"_A" for x in list(team_against)]
team_against

,Score_A,xScore_A,xT_created_A,xT_denied_A,xT_prevented_A,xT_received_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A
Opponent,,,,,,,,,,,,,,
Brisbane Lions,122,93.887604,0.106715,0.851522,0.221936,0.361493,24.260554,16.380475,22.310430,1.950125,121.078925,115.966759,114.265726,6.813198
Sydney,90,70.776025,-0.749615,0.221936,0.851522,-1.000685,19.236294,14.293043,16.395997,2.840297,100.570235,84.321734,87.444271,13.125965


Team Ratings - Combination of VAEP and Expected VAEP Values should be a correct order (Season by Season)

In [141]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.af

In [152]:
player_stats.groupby(['Match_ID', 'Team']).size().reset_index().groupby('Team').count()[['Match_ID']]

,Match_ID
Team,
Adelaide,62
Brisbane Lions,67
Carlton,63
Collingwood,65
Essendon,64
Fremantle,65
Geelong,68
Gold Coast,63
Greater Western Sydney,64


In [153]:
def create_team_ratings(player_stats):
    
    team_for = player_stats.groupby('Team').sum()[['Score', 'xScore', 'xT_created', 'xT_denied', 'xT_prevented', 'xT_received', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats.groupby(['Opponent']).sum()[['Score', 'xScore', 'xT_created', 'xT_denied', 'xT_prevented', 'xT_received', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats.groupby(['Match_ID', 'Team']).size().reset_index().groupby('Team').count()[['Match_ID']]
    team_games.columns = ['Games']
    team_ratings = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_ratings[col+"_pergame_F"] = team_ratings[col+"_F"] / team_ratings["Games"]
        team_ratings[col+"_pergame_A"] = team_ratings[col+"_A"] / team_ratings["Games"]

        team_ratings[col+"_diff"] = team_ratings[col+"_F"] - team_ratings[col+"_A"]
        team_ratings[col+"_pergame_diff"] = team_ratings[col+"_pergame_F"] - team_ratings[col+"_pergame_A"]

    return team_ratings


In [154]:
team_ratings = create_team_ratings(player_stats)
team_ratings[['Games', 'xScore_pergame_F', 'xScore_pergame_A', 'xScore_pergame_diff']].sort_values(by='xScore_pergame_diff', ascending=False)

,Games,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Melbourne,68,90.082720,69.766639,20.316081
Geelong,68,89.103006,71.379537,17.723470
Brisbane Lions,67,92.729587,77.005104,15.724482
Western Bulldogs,69,87.972055,75.400861,12.571194
Port Adelaide,65,86.507982,75.504758,11.003225
Sydney,66,88.601732,79.927198,8.674534
Carlton,63,86.855581,80.702845,6.152735
Richmond,64,86.178707,82.023348,4.155359
Collingwood,65,81.801971,78.482725,3.319246
Fremantle,65,80.594269,81.734468,-1.140199


In [156]:
team_ratings = create_team_ratings(player_stats[player_stats['Season']==2023])
team_ratings[['Games', 'xScore_pergame_F', 'xScore_pergame_A', 'xScore_pergame_diff', 'Score_pergame_F', 'Score_pergame_A', 'Score_pergame_diff']].sort_values(by='xScore_pergame_diff', ascending=False)

,Games,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff,Score_pergame_F,Score_pergame_A,Score_pergame_diff
Brisbane Lions,18,92.229738,72.714555,19.515182,88.833333,72.888889,15.944444
Port Adelaide,19,92.264866,73.963429,18.301438,90.263158,82.526316,7.736842
Collingwood,18,89.667063,75.044548,14.622514,90.944444,68.111111,22.833333
Melbourne,19,89.452364,75.277488,14.174876,89.842105,72.631579,17.210526
Carlton,19,88.134841,76.013819,12.121021,82.526316,70.631579,11.894737
Geelong,19,89.926954,79.305560,10.621393,89.894737,75.000000,14.894737
Adelaide,19,87.233396,77.564934,9.668462,93.842105,79.368421,14.473684
Sydney,18,89.141618,82.044436,7.097182,85.666667,78.888889,6.777778
Western Bulldogs,20,77.064257,72.664868,4.399389,81.250000,74.250000,7.000000
St Kilda,18,77.937276,75.780222,2.157054,75.944444,71.166667,4.777778


In [189]:
finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = player_stats['Round_ID'].str[4:].replace(finals_round_map).astype(int)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)


In [205]:
current_round_id = '202127'
current_season = int(current_round_id[:4])
current_round = int(current_round_id[4:])

current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round)]
# current_round_player_stats = player_stats[(player_stats['Round_ID_num'] <= current_round_id)]
current_round_team_ratings = create_team_ratings(current_round_player_stats)
current_round_team_ratings[['Games', 'xScore_pergame_F', 'xScore_pergame_A', 'xScore_pergame_diff']].sort_values(by='xScore_pergame_diff', ascending=False)

,Games,xScore_pergame_F,xScore_pergame_A,xScore_pergame_diff
Melbourne,25,92.907450,65.214994,27.692457
Western Bulldogs,26,93.294626,73.423622,19.871004
Brisbane Lions,24,94.685585,75.096850,19.588735
Port Adelaide,24,85.209355,73.136185,12.073171
Geelong,24,82.174535,70.883908,11.290627
Sydney,23,85.537569,80.282530,5.255039
Essendon,23,85.883458,83.188963,2.694495
Richmond,22,81.694916,83.022769,-1.327853
Greater Western Sydney,24,82.463687,84.763260,-2.299573
Carlton,22,84.732182,87.529614,-2.797432


In [198]:
current_round_id = '202323'
current_season = int(current_round_id[:4])
current_round = int(current_round_id[4:])

current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round)]
current_round_team_ratings = create_team_ratings(current_round_player_stats)
current_round_team_ratings[['Games', 'Score_F', 'Score_A', 'Score_diff', 'Score_pergame_F', 'Score_pergame_A', 'Score_pergame_diff']].sort_values(by='Score_pergame_diff', ascending=False)

,Games,Score_F,Score_A,Score_diff,Score_pergame_F,Score_pergame_A,Score_pergame_diff
Collingwood,18,1637,1226,411,90.944444,68.111111,22.833333
Melbourne,19,1707,1380,327,89.842105,72.631579,17.210526
Brisbane Lions,18,1599,1312,287,88.833333,72.888889,15.944444
Geelong,19,1708,1425,283,89.894737,75.000000,14.894737
Adelaide,19,1783,1508,275,93.842105,79.368421,14.473684
Carlton,19,1568,1342,226,82.526316,70.631579,11.894737
Port Adelaide,19,1715,1568,147,90.263158,82.526316,7.736842
Western Bulldogs,20,1625,1485,140,81.250000,74.250000,7.000000
Sydney,18,1542,1420,122,85.666667,78.888889,6.777778
St Kilda,18,1367,1281,86,75.944444,71.166667,4.777778


In [197]:
current_round_id = '202323'
current_season = int(current_round_id[:4])
current_round = int(current_round_id[4:])

current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round)]
current_round_team_ratings = create_team_ratings(current_round_player_stats)
current_round_team_ratings[['Games', 'vaep_value_F', 'vaep_value_A', 'vaep_value_diff', 'vaep_value_pergame_F', 'vaep_value_pergame_A', 'vaep_value_pergame_diff']].sort_values(by='vaep_value_pergame_diff', ascending=False)

,Games,vaep_value_F,vaep_value_A,vaep_value_diff,vaep_value_pergame_F,vaep_value_pergame_A,vaep_value_pergame_diff
Collingwood,18,334.974496,267.718233,67.256263,18.609694,14.873235,3.736459
Melbourne,19,364.704814,309.871402,54.833412,19.194990,16.309021,2.885969
Geelong,19,341.528822,293.547542,47.981280,17.975201,15.449871,2.525331
Brisbane Lions,18,337.745817,300.267590,37.478227,18.763656,16.681533,2.082124
Adelaide,19,357.933391,319.837486,38.095906,18.838600,16.833552,2.005048
Carlton,19,335.008596,300.711090,34.297506,17.632031,15.826899,1.805132
Port Adelaide,19,346.214001,329.243905,16.970096,18.221790,17.328627,0.893163
Sydney,18,329.339888,314.092665,15.247224,18.296660,17.449592,0.847068
Western Bulldogs,20,326.536916,314.424702,12.112214,16.326846,15.721235,0.605611
Greater Western Sydney,18,320.780465,312.711294,8.069171,17.821137,17.372850,0.448287


In [201]:
current_round_id = '202323'
current_season = int(current_round_id[:4])
current_round = int(current_round_id[4:])

current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round)]
current_round_team_ratings = create_team_ratings(current_round_player_stats)
current_round_team_ratings[['Games', 'exp_vaep_value_F', 'exp_vaep_value_A', 'exp_vaep_value_diff', 'exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']].sort_values(by='exp_vaep_value_pergame_diff', ascending=False)

,Games,exp_vaep_value_F,exp_vaep_value_A,exp_vaep_value_diff,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,19,2361.433553,2019.443506,341.990048,124.285976,106.286500,17.999476
Brisbane Lions,18,2139.106261,1820.798329,318.307931,118.839237,101.155463,17.683774
Melbourne,19,2251.679290,2009.093338,242.585952,118.509436,105.741755,12.767682
Collingwood,18,2090.668138,1862.468198,228.199940,116.148230,103.470455,12.677774
Carlton,19,2270.651941,2049.107988,221.543953,119.507997,107.847789,11.660208
Geelong,19,2176.850670,1966.709939,210.140731,114.571088,103.511049,11.060038
Adelaide,19,2254.852905,2072.620639,182.232266,118.676469,109.085297,9.591172
Sydney,18,2114.716497,1994.280316,120.436181,117.484250,110.793351,6.690899
Western Bulldogs,20,2177.163641,2087.126292,90.037349,108.858182,104.356315,4.501867
St Kilda,18,1979.870384,1915.963136,63.907248,109.992799,106.442396,3.550403
